In [11]:
import pandas as pd
from epilepsy_prediction import load_prediction_model, data_preprocess,fuse_string,evaluate_model
from copy import deepcopy

# How to predict

Loading the model, we can select between: Carbamazepine, Gabapentin, Lamotrigine, Levetiracetam, Oxcarbazepine, Phenytoin,Topiramate,Valproate. 

In [12]:
def load_data(change_name_file,data_file_name):
    data_df = pd.read_csv(data_file_name, index_col=0)
    new_name_df = pd.read_csv(change_name_file,index_col=0)
    data_df.rename(new_name_df['marketscan'].to_dict(),inplace=True,axis=1)
    y_train = data_df.loc[data_df["test_train"]=="train","outcome"]
    X_train = data_df.loc[data_df["test_train"]=="train",~( data_df.columns.isin(["outcome","test_train"]))]
    y_test = data_df.loc[~(data_df["test_train"]=="train"),"outcome"]
    X_test = data_df.loc[~(data_df["test_train"]=="train"),~ (data_df.columns.isin(["outcome","test_train"]))]
    return X_train,y_train,X_test,y_test


In [3]:
change_name_file=r"C:\\Users\Anastassia Kolde\\Documents\\Israel secondment\\dictionary.csv"
data_file_name = r"C:\\Users\Anastassia Kolde\\Documents\\Israel secondment\\Data\test_train_epilepsy.csv"
X_train,y_train,X_test,y_test = load_data(change_name_file,data_file_name)

In [4]:
drug_name="carbamazepine"
clf = load_prediction_model(drug_name)
X = data_preprocess(X_train,impute_drug=drug_name)
Xs = data_preprocess(X_test,impute_drug=drug_name)

Now we can use the model for prediction

And we can also use the model for training using it's fit method. 

In [6]:
new_clf = deepcopy(clf)
new_clf.fit(X,y_train)
new_clf.save_model('../models/prediction/est_carba.xgb')

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.3,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='logloss', gamma=0.4, gpu_id=-1,
              grow_policy='depthwise', importance_type='gain',
              interaction_constraints='', learning_rate=0.05, max_bin=256,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=4, max_leaves=0,
              min_child_weight=5, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, ...)

In [ ]:
eval = evaluate_model(clf,new_clf,Xs,y_test)
rep = fuse_string(eval)
print(rep)
